In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import yfinance as yf
import seaborn as sns
from datetime import datetime

In [2]:
# Import necessary libraries
from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract
from ibapi.order import Order
import datetime
import time


In [3]:
# Suppress all warnings
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [4]:
# Set the working directory
os.chdir(r"D:\Benson\aUpWork\Douglas Backtester Algo\Backtester Algorithm\Data\TWS API")

In [5]:
# Define the port number for TWS API connection
port = 7497  # Default port for TWS paper trading account

# master API account id
master_API_account_id = "254"


### CONNECTING TO TWS AND COLLECTING DATA THERE

In [7]:
# Create the App class - handles TWS connection and callbacks
class TradingApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = {}
        self.df = {}
        self.position = {}
        self.order_df = pd.DataFrame(columns=['PermId', 'ClientId', 'OrderId',
                                            'Account', 'Symbol', 'SecType',
                                            'Exchange', 'Action', 'OrderType',
                                            'TotalQty', 'CashQty', 'LmtPrice',
                                            'AuxPrice', 'Status'])
        
    def error(self, reqId, errorCode, errorString):
        print(f'Error {errorCode} {errorString}')
        
    def nextValidId(self, orderId):
        super().nextValidId(orderId)
        self.start()
        
    def start(self):
        print("Connected. Starting Trading App...")
        
    def historicalData(self, reqId, bar):
        if reqId not in self.data:
            self.data[reqId] = [{"Date": bar.date, "Open": bar.open, "High": bar.high,
                                "Low": bar.low, "Close": bar.close, "Volume": bar.volume}]
        else:
            self.data[reqId].append({"Date": bar.date, "Open": bar.open, "High": bar.high,
                                   "Low": bar.low, "Close": bar.close, "Volume": bar.volume})
        print(f"Historical Data: {bar}")

In [10]:
# Create connection function
def connect_tws():
    app = TradingApp()
    app.connect(host='127.0.0.1', port=port, clientId=master_API_account_id)  # port from previous cell
    print(f'Connecting to TWS on port {port}...')
    
    # Start the socket in a thread
    api_thread = threading.Thread(target=app.run, daemon=True)
    api_thread.start()
    
    # Wait for connection confirmation
    time.sleep(2)  # Give TWS time to respond
    
    return app

# Test the connection
if __name__ == '__main__':
    import threading
    app = connect_tws()
    print('Connection successful if no errors above')

Connecting to TWS on port 7497...
Connected. Starting Trading App...
Error 2104 Market data farm connection is OK:usfarm.nj
Error 2104 Market data farm connection is OK:usfuture
Error 2104 Market data farm connection is OK:cashfarm
Error 2104 Market data farm connection is OK:usfarm
Error 2106 HMDS data farm connection is OK:cashhmds
Error 2106 HMDS data farm connection is OK:ushmds
Error 2158 Sec-def data farm connection is OK:secdefil
Connection successful if no errors above


In [11]:
# Function to create a contract object
def create_contract(symbol, sec_type="STK", exchange="SMART", currency="USD"):
    contract = Contract()
    contract.symbol = symbol
    contract.secType = sec_type
    contract.exchange = exchange
    contract.currency = currency
    return contract

# Function to create an order object
def create_order(action, quantity, order_type="MKT"):
    order = Order()
    order.action = action
    order.totalQuantity = quantity
    order.orderType = order_type
    return order

In [12]:
# Enhanced TradingApp class with historical data handling
class TradingApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.data = []
        self.data_ready = False
        
    def error(self, reqId, errorCode, errorString):
        print(f'Error {errorCode} {errorString}')
        
    def historicalData(self, reqId, bar):
        self.data.append({
            'Date': pd.to_datetime(bar.date),
            'Open': bar.open,
            'High': bar.high,
            'Low': bar.low,
            'Close': bar.close,
            'Volume': bar.volume
        })
        
    def historicalDataEnd(self, reqId, start, end):
        print("Historical data download completed")
        self.data_ready = True

In [13]:
# Function to create YMAG contract
def create_ymag_contract():
    contract = Contract()
    contract.symbol = 'YMAG'
    contract.secType = 'STK'
    contract.exchange = 'SMART'
    contract.currency = 'USD'
    return contract

# Function to download historical data
def download_ymag_data(app, duration='1 Y', bar_size='1 hour'):
    contract = create_ymag_contract()
    
    # Reset the data container
    app.data = []
    app.data_ready = False
    
    # Request historical data
    app.reqHistoricalData(
        reqId=1,
        contract=contract,
        endDateTime='',  # Empty string means current time
        durationStr=duration,
        barSizeSetting=bar_size,
        whatToShow='TRADES',
        useRTH=1,
        formatDate=1,
        keepUpToDate=False,
        chartOptions=[]
    )
    
    # Wait for data to be ready
    while not app.data_ready:
        time.sleep(0.1)
    
    # Convert to DataFrame
    df = pd.DataFrame(app.data)
    df.set_index('Date', inplace=True)
    return df

In [14]:
# Connect and download data
if __name__ == '__main__':
    import threading
    
    # Connect to TWS
    app = connect_tws()
    time.sleep(1)  # Give time for connection to establish
    
    try:
        # Download YMAG data
        print("Downloading YMAG data...")
        ymag_df = download_ymag_data(app)
        
        # Display the first few rows
        print("\nFirst few rows of YMAG data:")
        print(ymag_df.head())
        
        # Save to CSV
        ymag_df.to_csv('YMAG_TWS_data.csv')
        print("\nData saved to YMAG_TWS_data.csv")
        
    finally:
        # Disconnect
        app.disconnect()
        print("Disconnected from TWS")

Connecting to TWS on port 7497...
Error 504 Not connected


: 

In [10]:
#!/usr/bin/env python3

import time
import threading
import csv

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

# -----------------------------------------------
# Custom Application Class
# -----------------------------------------------
class IBApp(EWrapper, EClient):
    def __init__(self):
        EClient.__init__(self, self)
        self.bars = []           # Will store the received bars here
        self.data_received = False

    def error(self, reqId, errorCode, errorString):
        """Called if there's an error with the request or connection."""
        print(f"Error. Id: {reqId}, Code: {errorCode}, Msg: {errorString}")

    def historicalData(self, reqId, bar):
        """
        Called for each bar of historical data.
        We'll collect these bars in a list for later CSV export.
        """
        # Store the bar data in a list (you can store as a dict or the BarData object)
        self.bars.append({
            "date": bar.date,
            "open": bar.open,
            "high": bar.high,
            "low": bar.low,
            "close": bar.close,
            "volume": bar.volume
        })

    def historicalDataEnd(self, reqId, start, end):
        """Called once all requested historical data has been received."""
        print("Historical Data End")
        self.data_received = True

# -----------------------------------------------
# Helper function to run the app in a thread
# -----------------------------------------------
def run_loop(app):
    app.run()

# -----------------------------------------------
# Main execution flow
# -----------------------------------------------
def main():
    # Connection details for TWS paper trading
    host = "127.0.0.1"
    port = 7497      # Default port for TWS paper trading
    client_id = 254

    # Create the app instance
    app = IBApp()
    # Connect to TWS
    app.connect(host, port, client_id)

    # Start the socket in a separate thread
    api_thread = threading.Thread(target=run_loop, args=(app,), daemon=True)
    api_thread.start()

    # Give the connection a moment to establish
    time.sleep(2)

    # Define the YMAG ETF contract
    contract = Contract()
    contract.symbol = "YMAG"
    contract.secType = "STK"       # ETFs in IB are generally handled as STK
    contract.currency = "USD"
    contract.exchange = "SMART"    # Or specify the primary listing exchange if needed

    # Request 1 month of daily historical data (adjust durationStr as desired)
    # endDateTime="" => current time
    app.reqHistoricalData(
        reqId=1,
        contract=contract,
        endDateTime="",
        durationStr="1 M",        # '1 M' = 1 month, '1 Y' = 1 year, etc.
        barSizeSetting="1 day",   # daily bars
        whatToShow="TRADES",      # could be 'TRADES', 'BID', 'ASK', 'MIDPOINT'
        useRTH=1,                 # 1 = regular trading hours only
        formatDate=1,             # 1 = YYYYMMDD format
        keepUpToDate=False,       # set True for streaming historical
        chartOptions=[]
    )

    # Wait until data is fully received
    while not app.data_received:
        time.sleep(1)

    # Disconnect from TWS
    app.disconnect()

    # -----------------------------------------------
    # Export data to CSV
    # -----------------------------------------------
    csv_filename = "YMAG_daily_data.csv"
    with open(csv_filename, mode="w", newline="") as csv_file:
        writer = csv.writer(csv_file)
        # Write header
        writer.writerow(["Date", "Open", "High", "Low", "Close", "Volume"])
        # Write each bar
        for bar in app.bars:
            writer.writerow([
                bar["date"],
                bar["open"],
                bar["high"],
                bar["low"],
                bar["close"],
                bar["volume"]
            ])

    print(f"Historical data saved to {csv_filename}")

if __name__ == "__main__":
    main()


Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:usfarm.nj
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:usfuture
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:cashfarm
Error. Id: -1, Code: 2104, Msg: Market data farm connection is OK:usfarm
Error. Id: -1, Code: 2106, Msg: HMDS data farm connection is OK:cashhmds
Error. Id: -1, Code: 2106, Msg: HMDS data farm connection is OK:ushmds
Error. Id: -1, Code: 2158, Msg: Sec-def data farm connection is OK:secdefil
Error. Id: 1, Code: 2176, Msg: Warning: Your API version does not support fractional share size rules. Please upgrade to a minimum version 163. Trimmed value 8735.37 to 8735
Historical Data End
Historical data saved to YMAG_daily_data.csv
